In [101]:
from goatools import obo_parser


In [2]:
!wget http://purl.obolibrary.org/obo/go/go-basic.obo

--2019-12-02 12:29:57--  http://purl.obolibrary.org/obo/go/go-basic.obo
Translacja purl.obolibrary.org (purl.obolibrary.org)... 52.3.123.63
Łączenie się z purl.obolibrary.org (purl.obolibrary.org)|52.3.123.63|:80... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 302 Found
Lokalizacja: http://current.geneontology.org/ontology/go-basic.obo [podążanie]
--2019-12-02 12:29:58--  http://current.geneontology.org/ontology/go-basic.obo
Translacja current.geneontology.org (current.geneontology.org)... 54.192.230.66, 54.192.230.85, 54.192.230.20, ...
Łączenie się z current.geneontology.org (current.geneontology.org)|54.192.230.66|:80... połączono.
Żądanie HTTP wysłano, oczekiwanie na odpowiedź... 200 OK
Długość: 31285115 (30M) [text/obo]
Zapis do: `go-basic.obo'

go-basic.obo        100%[===================>]  29,83M  1,85MB/s     w 16s     

2019-12-02 12:30:14 (1,84 MB/s) - zapisano `go-basic.obo' [31285115/31285115]



In [102]:
go_obo = 'go-basic.obo'

In [103]:
# Now we can create a dictionary of the GO terms, 
# using the obo_parser from GOATools
go = obo_parser.GODag(go_obo)

go-basic.obo: fmt(1.2) rel(2019-11-02) 47,242 GO Terms


In [12]:
go_id = 'GO:0048527'
go_term = go[go_id]
print(go_term)

GO:0048527	level-05	depth-06	lateral root development [biological_process]


In [13]:
print(go_term.name)

lateral root development


In [18]:
parents = go_term.get_all_parents()
print(parents)

{'GO:0048364', 'GO:0090696', 'GO:0008150', 'GO:0048856', 'GO:0048528', 'GO:0009791', 'GO:0032502', 'GO:0032501', 'GO:0099402'}


In [20]:
for term in parents:
    print(go[term])

GO:0048364	level-04	depth-04	root development [biological_process]
GO:0090696	level-03	depth-04	post-embryonic plant organ development [biological_process]
GO:0008150	level-00	depth-00	biological_process [biological_process]
GO:0048856	level-02	depth-02	anatomical structure development [biological_process]
GO:0048528	level-04	depth-05	post-embryonic root development [biological_process]
GO:0009791	level-02	depth-02	post-embryonic development [biological_process]
GO:0032502	level-01	depth-01	developmental process [biological_process]
GO:0032501	level-01	depth-01	multicellular organismal process [biological_process]
GO:0099402	level-03	depth-03	plant organ development [biological_process]


In [21]:
lineage_png = go_id + '-lineage.png'

In [22]:
go.draw_lineage([go_term], lineage_img=lineage_png)

lineage info for terms ['GO:0048527'] written to GO:0048527-lineage.png


## QuickGO

In [23]:
from urllib.request import urlopen
import json

def get_term(go_id):
    """
        This function retrieves the definition of a given Gene Ontology term,
        using EMBL-EBI's QuickGO browser.
        Input: go_id - a valid Gene Ontology ID, e.g. GO:0048527.
    """
    quickgo_url = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/terms/" + go_id
    ret = urlopen(quickgo_url)
    
    # Check the response
    if(ret.getcode() == 200):
        term = json.loads(ret.read())
        return term['results'][0]
    else:
        raise ValueError("Couldn't receive information from QuickGO. Check GO ID and try again.")

In [25]:
go_id = 'GO:0048527'
term = get_term(go_id)
term

{'id': 'GO:0048527',
 'isObsolete': False,
 'name': 'lateral root development',
 'definition': {'text': 'The process whose specific outcome is the progression of the lateral root over time, from its formation to the mature structure. A lateral root is one formed from pericycle cells located on the xylem radius of the root, as opposed to the initiation of the main root from the embryo proper.'},
 'children': [{'id': 'GO:1902089', 'relation': 'part_of'},
  {'id': 'GO:1901332', 'relation': 'negatively_regulates'},
  {'id': 'GO:2000023', 'relation': 'regulates'},
  {'id': 'GO:0010102', 'relation': 'part_of'},
  {'id': 'GO:1901333', 'relation': 'positively_regulates'}],
 'aspect': 'biological_process',
 'usage': 'Unrestricted'}

In [26]:
print('Description of {} is:\n\t{}'.format(go_id, term['definition']['text']))

Description of GO:0048527 is:
	The process whose specific outcome is the progression of the lateral root over time, from its formation to the mature structure. A lateral root is one formed from pericycle cells located on the xylem radius of the root, as opposed to the initiation of the main root from the embryo proper.


## section 3 - GAF (Gene Association File)

In [85]:
import Bio.UniProt.GOA as GOA

salmo_gaf = '/mnt/sdb1/goa_uniprot_salmo.gaf'

# File is a gunzip file, so we need to open it in this way
with open(salmo_gaf, 'rt') as salmo_gaf_fp:
    
    salmo_funcs = {}  # Initialise the dictionary of functions
    
    # Iterate on each function using Bio.UniProt.GOA library.
    for entry in GOA.gafiterator(salmo_gaf_fp):
        salmo_id = entry.pop('DB_Object_ID') #usuwa klucz z gaf zapisuje go jako salmo_id
        salmo_funcs[salmo_id] = entry #dodaje usuniety klucz jako klucz w nowym slowniku i dodaje do niego pozostale klucze ze slownika gaf (entry)
salmo_funcs

{'A0A088N9E5': {'DB': 'UniProtKB',
  'DB_Object_Symbol': 'A0A088N9E5',
  'Qualifier': [''],
  'GO_ID': 'GO:0016021',
  'DB:Reference': ['GO_REF:0000038'],
  'Evidence': 'IEA',
  'With': ['UniProtKB-KW:KW-0812'],
  'Aspect': 'C',
  'DB_Object_Name': 'Transmembrane 50B-like protein',
  'Synonym': [''],
  'DB_Object_Type': 'protein',
  'Taxon_ID': ['taxon:8030'],
  'Date': '20191109',
  'Assigned_By': 'UniProt',
  'Annotation_Extension': '',
  'Gene_Product_Form_ID': ''},
 'A0A0A7MA29': {'DB': 'UniProtKB',
  'DB_Object_Symbol': 'A0A0A7MA29',
  'Qualifier': [''],
  'GO_ID': 'GO:0016021',
  'DB:Reference': ['GO_REF:0000038'],
  'Evidence': 'IEA',
  'With': ['UniProtKB-KW:KW-0812'],
  'Aspect': 'C',
  'DB_Object_Name': 'G_PROTEIN_RECEP_F1_2 domain-containing protein',
  'Synonym': [''],
  'DB_Object_Type': 'protein',
  'Taxon_ID': ['taxon:8030'],
  'Date': '20191109',
  'Assigned_By': 'UniProt',
  'Annotation_Extension': '',
  'Gene_Product_Form_ID': ''},
 'A0A0G3B776': {'DB': 'UniProtKB',
 

In [88]:
print(len(salmo_funcs))

14501


In [82]:
# wyszukiwanie specyficznych nazw

import re

growth_dict = {x: salmo_funcs[x]
               for x in salmo_funcs 
               if re.findall('NUP53', salmo_funcs[x]['DB_Object_Name'])}

In [83]:
print('UniProt IDs of annotations with "" in their name:')
for annot in growth_dict:
    print("\t - " + annot)
print("Total: {}".format(len(growth_dict)))

UniProt IDs of annotations with "" in their name:
	 - A0A1S3P0T0
	 - A0A1S3PR00
	 - A0A1S3PSB3
Total: 3


## Section 4 - GO enrichment or depletion analysis

In [97]:
from goatools.go_enrichment import GOEnrichmentStudy

Perform an enrichment analysis using the list of genes with the "growth" keyword from exercise 3.1.c, and the GO structure from exercise 2.1.

In [91]:
# The population is the functions observed in the Arabidopsis thaliana GOA file.
pop = salmo_funcs.keys()
len(pop)

14501

In [93]:
# Then, we need to create a dictionary of genes with 
# their UniProt ID as a key and their set of GO annotations as the values.


assoc = {}

for x in salmo_funcs:
    if x not in assoc:
        assoc[x] = set()
    assoc[x].add(str(salmo_funcs[x]['GO_ID']))

assoc

{'A0A088N9E5': {'GO:0016021'},
 'A0A0A7MA29': {'GO:0016021'},
 'A0A0G3B776': {'GO:0055114'},
 'A0A0G3BAW3': {'GO:0055114'},
 'A0A0G3BCK8': {'GO:0055114'},
 'A0A0R5T2D3': {'GO:0016021'},
 'A0A0R5T2D5': {'GO:0016021'},
 'A0A0R5T2D7': {'GO:0016021'},
 'A0A0R5T2E0': {'GO:0016021'},
 'A0A0R5T2E1': {'GO:0016021'},
 'A0A0R5T2E2': {'GO:0016021'},
 'A0A0R5T2E3': {'GO:0016021'},
 'A0A0R5T2E4': {'GO:0016021'},
 'A0A0R5T2E5': {'GO:0016021'},
 'A0A0R5T2E6': {'GO:0016021'},
 'A0A0R5T2F0': {'GO:0016021'},
 'A0A0R5T2F1': {'GO:0016021'},
 'A0A0R5T2F3': {'GO:0016021'},
 'A0A0R5T2F4': {'GO:0016021'},
 'A0A0R5T2F5': {'GO:0016021'},
 'A0A0R5T2F6': {'GO:0016021'},
 'A0A0R5T2F7': {'GO:0016021'},
 'A0A0R5T2F9': {'GO:0016021'},
 'A0A0R5T2G0': {'GO:0016021'},
 'A0A0R5T2G1': {'GO:0016021'},
 'A0A0R5T2G2': {'GO:0016021'},
 'A0A0R5T2G3': {'GO:0016021'},
 'A0A0R5T2G5': {'GO:0016021'},
 'A0A0R5T2G6': {'GO:0016021'},
 'A0A0R5T2G7': {'GO:0016021'},
 'A0A0R5T2G8': {'GO:0016021'},
 'A0A0R5T2G9': {'GO:0016021'},
 'A0A0R5

In [ ]:
# Now, the study set here is those genes with 
# the "" keyword, found previously.

In [95]:
study = growth_dict.keys()
study

dict_keys(['A0A1S3P0T0', 'A0A1S3PR00', 'A0A1S3PSB3'])

Which GO term is most significantly enriched or depleted? Does this make sense?

In [104]:
methods = ["bonferroni", "sidak", "holm", "fdr"]

g = GOEnrichmentStudy(pop, assoc, go,
                         propagate_counts=True,
                         alpha=0.05,
                         methods=methods)
g_res = g.run_study(study)


Load GOEA Gene Ontology Analysis ...
fisher module not installed.  Falling back on scipy.stats.fisher_exact
Propagating term counts up: is_a
100% 14,501 of 14,501 population items found in association

Run GOEA Gene Ontology Analysis: current study set of 3 IDs ...
100%      3 of      3 study items found in association
100%      3 of      3 study items found in population(14501)
Calculating 1,511 uncorrected p-values using fisher_scipy_stats
   1,511 GO terms are associated with 14,501 of 14,501 population items
      11 GO terms are associated with      3 of      3 study items
  METHOD bonferroni:
      10 GO terms found significant (< 0.05=alpha) ( 10 enriched +   0 purified): local bonferroni
       3 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)
  METHOD sidak:
      10 GO terms found significant (< 0.05=alpha) ( 10 enriched +   0 purified): local sidak
       3 study items associated with significant G

Generate p-value distribution for FDR based on resampling (this might take a while)
Sample 0 / 500: p-value 0.5604231106699784
Sample 10 / 500: p-value 0.04422800352283137
Sample 20 / 500: p-value 0.0018609134436252566
Sample 30 / 500: p-value 0.055425998751221194
Sample 40 / 500: p-value 0.10281210524298455
Sample 50 / 500: p-value 0.055425998751221194
Sample 60 / 500: p-value 0.5604231106699784
Sample 70 / 500: p-value 0.5604231106699784
Sample 80 / 500: p-value 0.055425998751221194
Sample 90 / 500: p-value 0.06475652748842697
Sample 100 / 500: p-value 0.001240865710934801
Sample 110 / 500: p-value 0.5604231106699784
Sample 120 / 500: p-value 0.5604231106699784
Sample 130 / 500: p-value 0.07441846426334721
Sample 140 / 500: p-value 0.07441846426334721
Sample 150 / 500: p-value 0.0006205612474475628
Sample 160 / 500: p-value 0.5604231106699784
Sample 170 / 500: p-value 0.0018609134436252566
Sample 180 / 500: p-value 0.07441846426334721
Sample 190 / 500: p-value 0.3285721497666471
Samp

  METHOD fdr:
      10 GO terms found significant (< 0.05=alpha) ( 10 enriched +   0 purified): local fdr
       3 study items associated with significant GO IDs (enriched)
       0 study items associated with significant GO IDs (purified)


Sample 490 / 500: p-value 0.07441846426334721


In [114]:
g.wr_xlsx("results.xlsx", g_res)


   1511 items WROTE: results.xlsx


In [ ]:
g_res.pri